In [1]:
using GeometryBasics: Vec3f, Point3f
using LinearAlgebra
using GLMakie
using StaticArrays
using VMRobotControl
using VMRobotControl.Splines: CubicSpline
using DifferentialEquations
using MeshIO

## Importing ShadowHand URDF

### URDF Parsing

In [2]:
using FileIO, UUIDs
try
    FileIO.add_format(format"DAE", (), ".dae", [:DigitalAssetExchangeFormatIO => UUID("43182933-f65b-495a-9e05-4d939cea427d")])
catch
end

cfg = URDFParserConfig(;suppress_warnings=true) # This is just to hide warnings about unsupported URDF features
module_path = joinpath(splitpath(splitdir(pathof(VMRobotControl))[1])[1:end-1])
robot = parseURDF(joinpath(module_path, "URDFs/sr_description/sr_hand_vm_compatible.urdf"), cfg)

24DOF Mechanism{Float64} "shadowhand_motor" with 33 frames, 32 joints, 55 coordinates, 82 components

### Definition of the Path

In [15]:
display(methods(Rail))
display(methods(CubicSpline))

# 2 methods for generic function "Rail" from VMRobotControl:
 [1] Rail(spline::Union{CubicSpline{3, T}, VMRobotControl.Splines.LoopingCubicSpline{3, T}}) where T
     @ C:\Users\bilou\OneDrive\Documents\Ecole\MASTER 2\TFE\Julia Simulator\VMRobotControl.jl\src\joints\joint_definitions.jl:167
 [2] Rail(spline::Union{CubicSpline{3, T}, VMRobotControl.Splines.LoopingCubicSpline{3, T}}, transform::Transform{T}) where T
     @ C:\Users\bilou\OneDrive\Documents\Ecole\MASTER 2\TFE\Julia Simulator\VMRobotControl.jl\src\joints\joint_definitions.jl:161

# 2 methods for type constructor:
 [1] CubicSpline(data::VMRobotControl.Splines.CubicSplineData{D, T}) where {D, T}
     @ C:\Users\bilou\OneDrive\Documents\Ecole\MASTER 2\TFE\Julia Simulator\VMRobotControl.jl\src\splines.jl:251
 [2] CubicSpline(knots)
     @ C:\Users\bilou\OneDrive\Documents\Ecole\MASTER 2\TFE\Julia Simulator\VMRobotControl.jl\src\splines.jl:265

In [3]:
start = SVector(0.0, -0.2, 0.32)

s_length = 20e-2
Nk = 6
L = 0.1
y0 = start[2]
y_vector = Vector(LinRange(y0, y0+L, Nk))
spline_points = Matrix{Float64}(undef, 0, 3)
for i = 1:Nk
    spline_points = vcat(
        spline_points,
        hcat(start[1], y_vector[i], start[3])
    )
end

spline = CubicSpline(spline_points)

CubicSpline{3, Float64}(VMRobotControl.Splines.CubicSplineData{3, Float64}([0.0 -0.2 0.32; 0.0 -0.18000000000000005 0.32; … ; 0.0 -0.12000000000000001 0.32; 0.0 -0.1 0.32], [0.0 -0.2 0.32; 0.0 -0.1800000000000001 0.31999999999999995; … ; 0.0 -0.12000000000000002 0.31999999999999995; 0.0 -0.1 0.32]))

### Hand Visualization

In [11]:
# Compile robot model
m = compile(robot)
kcache = Observable(new_kinematics_cache(m))  # This stores the robot's joint state

# Create the figure
fig = Figure(size=(800, 600))

# Create interactive 3D scene
ls = LScene(fig[1, 1]; show_axis=true)  # 3D interactive scene

# Attach interactive camera controls
cam3d!(ls)  # Enables mouse interaction (rotate, zoom, pan)

# Plot the robot's visuals
robotvisualize!(ls, kcache)

# Display figure
display(fig)

GLMakie.Screen(...)

## Creating the Virtual Mechanism System

### Gravity Compensation and Joint Damping

In [4]:
# GRAVITY COMPENSATION

add_gravity_compensation!(robot, VMRobotControl.DEFAULT_GRAVITY)

# JOINT DAMPING and LIMIT SPRINGS

joint_limits = cfg.joint_limits

for joint_id in keys(joints(robot))
    limits = joint_limits[joint_id]
    isnothing(limits) && continue
    add_coordinate!(robot, JointSubspace(joint_id);  id="$(joint_id)_coord")
    @assert ~isnothing(limits.lower) && ~isnothing(limits.upper)
    add_deadzone_springs!(robot, 50.0, (limits.lower+0.1, limits.upper-0.1), "$(joint_id)_coord")
    add_component!(robot, LinearDamper(0.01, "$(joint_id)_coord"); id="$(joint_id)_damper")
end

### Virtual Mechanism Initialization

In [5]:
vm = Mechanism{Float64}("VirtualTrack")

cart_frame = add_frame!(vm, "Cart")
add_joint!(vm, Rail(spline, zero(Transform{Float64}));
        parent=root_frame(vm), child=cart_frame,        id="RailJoint")

add_coordinate!(vm, JointSubspace("RailJoint");         id="CartDistance")
add_coordinate!(vm, FrameOrigin(cart_frame);            id="CartPosition")
add_component!(vm, LinearInerter(1.0, "CartPosition");  id="CartInertance") # Cart mass
add_component!(vm, LinearDamper(100.0, "CartPosition"); id="CartDamper");

In [13]:
vms = VirtualMechanismSystem("myShadowVMS", robot, vm)
root = root_frame(vms.robot)

add_coordinate!(vms, CoordDifference(".robot.rh_fftip_mass_coord", ".virtual_mechanism.CartPosition"); id="ff position error")
add_coordinate!(vms, CoordDifference(".robot.rh_mftip_mass_coord", ".virtual_mechanism.CartPosition"); id="mf position error")
add_coordinate!(vms, CoordDifference(".robot.rh_rftip_mass_coord", ".virtual_mechanism.CartPosition"); id="rf position error")
add_coordinate!(vms, CoordDifference(".robot.rh_lftip_mass_coord", ".virtual_mechanism.CartPosition"); id="lf position error")
add_coordinate!(vms, CoordDifference(".robot.rh_thtip_mass_coord", ".virtual_mechanism.CartPosition"); id="th position error")

K = 100.0 * identity(3)
D = 30.0 * identity(3)
add_component!(vms, LinearSpring(K, "ff position error");     id="ffSpring")
add_component!(vms, LinearDamper(D, "ff position error");     id="ffDamper")
add_component!(vms, LinearSpring(K, "mf position error");     id="mfSpring")
add_component!(vms, LinearDamper(D, "mf position error");     id="mfDamper")
add_component!(vms, LinearSpring(K, "rf position error");     id="rfSpring")
add_component!(vms, LinearDamper(D, "rf position error");     id="rfDamper")
add_component!(vms, LinearSpring(K, "lf position error");     id="lfSpring")
add_component!(vms, LinearDamper(D, "lf position error");     id="lfDamper")
add_component!(vms, LinearSpring(K, "th position error");     id="thSpring")
add_component!(vms, LinearDamper(D, "th position error");     id="thDamper")


max_power = 10.0
force_source = ForceSource(SVector(2.0), max_power, ".virtual_mechanism.CartDistance")
add_component!(vms, force_source;   id="Force source");

## Simulating the Robot

### Setting Up the Simulation

In [7]:
using Logging: global_logger
using TerminalLoggers: TerminalLogger
global_logger(TerminalLogger())

Logging.ConsoleLogger(IJulia.IJuliaStdio{Base.PipeEndpoint}(IOContext(Base.PipeEndpoint(Base.Libc.WindowsRawSocket(0x00000000000003ac) open, 0 bytes waiting))), Info, Logging.default_metafmt, true, 0, Dict{Any, Int64}())

In [14]:
tspan = (0., 5.)
vms_compiled = compile(vms)
q = (zero_q(vms_compiled.robot), [0.0]) # Robot joint angle, vm joint angles
q̇ = (zero_q̇(vms_compiled.robot), zeros(1)) # Robot joint velocity, vm joint velocities
g = VMRobotControl.DEFAULT_GRAVITY
dcache = new_dynamics_cache(vms_compiled)
prob = get_ode_problem(dcache, g, q, q̇, tspan)
@info "Simulating shadow robot with path following"

sol = solve(prob, Tsit5(), progress=true; maxiters=1e6, abstol=1e-6, reltol=1e-6);

[ Info: Simulating shadow robot with path following
ODE   0%|█                                              |  ETA: N/A
ODE   2%|█                                              |  ETA: 0:00:52
ODE   4%|██                                             |  ETA: 0:00:44
ODE   6%|███                                            |  ETA: 0:00:41
ODE   9%|█████                                          |  ETA: 0:00:39
ODE  11%|██████                                         |  ETA: 0:00:38
ODE  13%|███████                                        |  ETA: 0:00:36
ODE  16%|████████                                       |  ETA: 0:00:36
ODE  18%|█████████                                      |  ETA: 0:00:35
ODE  20%|██████████                                     |  ETA: 0:00:34
ODE  23%|███████████                                    |  ETA: 0:00:33
ODE  25%|████████████                                   |  ETA: 0:00:32
ODE  27%|█████████████                                  |  ETA: 0:00:31
ODE  30%|███████

### Visualizing the Results

In [21]:
plotting_t = Observable(0.0)
plotting_kcache = Observable(new_kinematics_cache(compile(vms)))
plotting_vm_kcache = map(plotting_kcache) do k
    VMRobotControl.virtual_mechanism_cache(k)
end
cartID = get_compiled_coordID(plotting_kcache[], ".virtual_mechanism.CartPosition")

fig = Figure(; size=(720, 720), figure_padding=0)
display(fig)
ls = LScene(fig[1, 1]; show_axis=false)
cam = cam3d!(ls; center=false)
cam.lookat[] = [0.25, 0.5, 0.1]
cam.eyeposition[] = [-0.4, -0.8, 0.5]

scatter!(ls, plotting_kcache, cartID; color=:red, marker=:rect, markersize=5)
robotvisualize!(ls, plotting_kcache)
robotsketch!(ls, plotting_vm_kcache)

fps = 60
T = sol.t[end]
N_frames = Int(floor(fps * T))
ts = LinRange(0.0, T, N_frames)
savepath = joinpath(module_path, "docs/src/assets/shadowhand_path_following.mp4")
animate_robot_odesolution(fig, sol, plotting_kcache, savepath; t=plotting_t);